In [3]:
from lumibot.brokers import Alpaca
from lumibot.backtesting import YahooDataBacktesting
from lumibot.strategies.strategy import Strategy
from lumibot.traders import Trader
from datetime import datetime
from alpaca_trade_api import REST
from timedelta import Timedelta

In [4]:
API_KEY = "PK1KAWB732Z95VFMPIEB"
API_SECRET = "lRxdiVh4jOoyZQvgUdqplHNuQZWI9apIPIXPaoaM"
BASE_URL = "https://paper-api.alpaca.markets"
ALPACA_CREDS = {
    "API_KEY":API_KEY,
    "API_SECRET":API_SECRET,
    "PAPER": True
}

In [9]:


class MLTrader(Strategy):
    def initialize(self, symbol:str="GOOGL", cash_at_risk:float = .5):
        self.cash_at_risk = cash_at_risk
        self.symbol = symbol
        self.sleeptime = "24H"
        self.last_trade = None
        self.api = REST(base_url=BASE_URL, 
                        key_id=API_KEY, 
                        secret_key=API_SECRET)
    
    def position_sizing(self):
        cash = self.get_cash()
        last_price = self.get_last_price(self.symbol)
        quantity = round(cash * self.cash_at_risk / last_price, 0)

        return cash, last_price, quantity
    
    def get_dates(self):
        today = self.get_datetime()
        three_days_prior = today - Timedelta(days = 3)
        return today.strftime('%Y-%m-%d'), three_days_prior.strftime('%Y-%m-%d')
    
    def get_news(self):
        today, three_days_prior = self.get_dates()
        news = self.api.get_news(symbol=self.symbol, 
                                 start = three_days_prior, 
                                 end = today )
        
        news = [ev.__dict__["_raw"]["headline"] for ev in news]
        return news


    def on_trading_iteration(self):
        cash, last_price, quantity = self.position_sizing()
        if cash > last_price:
            if self.last_trade == None:
                news = self.get_news()
                print(news)
                order = self.create_order(
                    self.symbol,
                    quantity,
                    "buy",
                    type="bracket",
                    take_profit_price=last_price*1.20,
                    stop_loss_price=last_price*.95
                )
                self.submit_order(order)
                self.last_trade = "buy"
        


brokers = Alpaca(ALPACA_CREDS)
strategy = MLTrader(name='mlstrat', broker=brokers, parameters={"symbol": "GOOGL", 
                                                                "cash_at_risk":.5})

start_date = datetime(2023,12,15)
end_date = datetime(2023,12,31)

strategy.backtest(
    YahooDataBacktesting,
    start_date,
    end_date,
    parameters={"symbol": "GOOGL", 
                "cash_at_risk":.5}
)

Starting backtest for MLTrader...
Progress ||   2.47%  [Elapsed: 0:00:01 ETA: 0:00:57] ["SpaceX Worth $180 Billion Now: Musk-Owned Company's Value Tops Boeing, Lockheed Martin, Disney, Verizon, IBM, Pfizer,  UPS", "'Google Plans New 'Pixie' AI Assistant for Pixel Phones' - The Information", "Tech Meets Tune: Universal Music's Lucian Grainge's Insights into AI's Resonance in the Melody of Tomorrow And How To Invest In The Soundscape", "Google's AI Search - A Double-Edged Sword for Publishers Like the Atlantic", 'Market Clubhouse Morning Memo - December 14th, 2023 (Trade Strategy For SPY, QQQ, AAPL, MSFT, TSLA, NVDA, GOOGL, And META)', "Can Alphabet's Gemini AI Catapult Google Ahead in Tech Race?", 'European Commission Sends Request For Information To Apple And Google Under The Digital Services Act', "Want To Repair Your iPhone Or MacBook At Home? Apple's New Diagnostic Tool Makes It Easy", 'Google Is Showering Discounts On Pixel 8 Pro: Now YouTube Premium Subscribers Get Flat $325 Off',

{'cagr': 1.0753092333495213,
 'volatility': 0.09183644996007587,
 'sharpe': 11.13892400269676,
 'max_drawdown': {'drawdown': 0.007176093998772082,
  'date': Timestamp('2023-12-29 09:30:00-0500', tz='America/New_York')},
 'romad': 149.84603511792346,
 'total_return': 0.03043803710937465}

## ML Components for news (Sentimental Analysis)

In [ ]:
# pip install alpaca-trade-api==3.1.1
# pip install timedelta